<a href="https://colab.research.google.com/github/tasmimjoti/Computer-Vision/blob/main/Brain_Tumor_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import (Conv2D, MaxPooling2D, Flatten, Dense,
                                     Dropout, BatchNormalization, Input, GlobalAveragePooling2D)
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import VGG16, ResNet50, InceptionV3
from tensorflow.keras.applications.vgg16 import preprocess_input as vgg_preprocess
from tensorflow.keras.applications.resnet50 import preprocess_input as resnet_preprocess
from tensorflow.keras.applications.inception_v3 import preprocess_input as inception_preprocess
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
train_dir = '/content/drive/MyDrive/Colab Notebooks/Brain Tumor Detection/Training'
test_dir = '/content/drive/MyDrive/Colab Notebooks/Brain Tumor Detection/Testing'

IMG_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS = 50

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.2,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

class_names = list(train_generator.class_indices.keys())

Found 5722 images belonging to 4 classes.
Found 1311 images belonging to 4 classes.


In [ ]:
def evaluate_model(model, test_generator, model_name, class_names):
    print(f"\n--- Evaluation for {model_name} ---")
    Y_pred = model.predict(test_generator)
    y_pred = np.argmax(Y_pred, axis=1)
    y_true = test_generator.classes

    print("\nClassification Report:\n")
    print(classification_report(y_true, y_pred, target_names=class_names))

    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(8,6))
    sns.heatmap(cm, annot=True, fmt='d', xticklabels=class_names, yticklabels=class_names, cmap='Blues')
    plt.title(f'{model_name} - Confusion Matrix')
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.show()

def show_sample_predictions(model, generator, class_names, num_samples=5):
    x_test, y_test = next(generator)
    y_pred = model.predict(x_test)
    y_pred_classes = np.argmax(y_pred, axis=1)
    y_true_classes = np.argmax(y_test, axis=1)

    plt.figure(figsize=(15, 5))
    for i in range(num_samples):
        plt.subplot(1, num_samples, i+1)
        plt.imshow(x_test[i])
        true_label = class_names[y_true_classes[i]]
        pred_label = class_names[y_pred_classes[i]]
        color = 'green' if true_label == pred_label else 'red'
        plt.title(f"True: {true_label}\nPred: {pred_label}", color=color)
        plt.axis('off')
    plt.tight_layout()
    plt.show()

def plot_history(history, model_name):
    plt.figure(figsize=(12, 4))

    # Accuracy
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Train')
    plt.plot(history.history['val_accuracy'], label='Val')
    plt.title(f'{model_name} - Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()

    # Loss
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Train')
    plt.plot(history.history['val_loss'], label='Val')
    plt.title(f'{model_name} - Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()

    plt.show()

In [ ]:
def create_custom_cnn(input_shape=(224, 224, 3), num_classes=4):
    model = Sequential([
        Conv2D(16, (3, 3), activation='relu', input_shape=input_shape),
        MaxPooling2D(2, 2),
        BatchNormalization(),

        Conv2D(32, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),
        BatchNormalization(),

        Flatten(),
        Dense(64, activation='relu'),
        Dropout(0.3),
        Dense(num_classes, activation='softmax')
    ])
    return model

custom_cnn = create_custom_cnn(input_shape=IMG_SIZE + (3,))
custom_cnn.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

history_custom = custom_cnn.fit(train_generator, validation_data=test_generator, epochs=EPOCHS)
plot_history(history_custom, 'CNN')
evaluate_model(custom_cnn, test_generator, 'CNN', class_names)
show_sample_predictions(custom_cnn, test_generator, class_names)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/50
179/179 ━━━━━━━━━━━━━━━━━━━━ 1319s 7s/step - accuracy: 0.5046 - loss: 3.0173 - val_accuracy: 0.2311 - val_loss: 1.3866
Epoch 2/50
179/179 ━━━━━━━━━━━━━━━━━━━━ 384s 2s/step - accuracy: 0.5716 - loss: 1.0849 - val_accuracy: 0.2342 - val_loss: 1.3909
Epoch 3/50
179/179 ━━━━━━━━━━━━━━━━━━━━ 385s 2s/step - accuracy: 0.5767 - loss: 1.1310 - val_accuracy: 0.4622 - val_loss: 1.3098
Epoch 4/50
179/179 ━━━━━━━━━━━━━━━━━━━━ 377s 2s/step - accuracy: 0.6270 - loss: 0.9701 - val_accuracy: 0.7185 - val_loss: 0.7556
Epoch 5/50
179/179 ━━━━━━━━━━━━━━━━━━━━ 387s 2s/step - accuracy: 0.6476 - loss: 0.8628 - val_accuracy: 0.7155 - val_loss: 0.7816
Epoch 6/50
179/179 ━━━━━━━━━━━━━━━━━━━━ 407s 2s/step - accuracy: 0.6688 - loss: 0.8584 - val_accuracy: 0.6834 - val_loss: 0.9656
Epoch 7/50
179/179 ━━━━━━━━━━━━━━━━━━━━ 399s 2s/step - accuracy: 0.6820 - loss: 0.8133 - val_accuracy: 0.7613 - val_loss: 0.7275
Epoch 8/50
179/179 ━━━━━━━━━━━━━━━━━━━━ 394s 2s/step - accuracy: 0.6627 - loss: 0.7971 - val_acc

In [ ]:

def create_transfer_model(base_model_class, preprocess_fn, input_shape=(224, 224, 3), num_classes=4):
    base_model = base_model_class(include_top=False, input_shape=input_shape, weights='imagenet')
    base_model.trainable = False

    inputs = Input(shape=input_shape)
    x = preprocess_fn(inputs)
    x = base_model(x, training=False)
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.5)(x)
    outputs = Dense(num_classes, activation='softmax')(x)

    return Model(inputs, outputs)

In [ ]:
vgg_model = create_transfer_model(VGG16, vgg_preprocess)
vgg_model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

history_vgg = vgg_model.fit(train_generator, validation_data=test_generator, epochs=EPOCHS)
plot_history(history_vgg, 'VGG16')
evaluate_model(vgg_model, test_generator, 'VGG16', class_names)
show_sample_predictions(vgg_model, test_generator, class_names)

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
Epoch 1/50
179/179 ━━━━━━━━━━━━━━━━━━━━ 158s 786ms/step - accuracy: 0.2678 - loss: 1.7568 - val_accuracy: 0.3799 - val_loss: 1.2951
Epoch 2/50
179/179 ━━━━━━━━━━━━━━━━━━━━ 119s 665ms/step - accuracy: 0.3478 - loss: 1.3019 - val_accuracy: 0.3524 - val_loss: 1.2723
Epoch 3/50
179/179 ━━━━━━━━━━━━━━━━━━━━ 114s 636ms/step - accuracy: 0.3686 - loss: 1.2646 - val_accuracy: 0.4119 - val_loss: 1.2594
Epoch 4/50
179/179 ━━━━━━━━━━━━━━━━━━━━ 113s 628ms/step - accuracy: 0.3722 - loss: 1.2646 - val_accuracy: 0.3814 - val_loss: 1.2531
Epoch 5/50
179/179 ━━━━━━━━━━━━━━━━━━━━ 115s 643ms/step - accuracy: 0.3823 - loss: 1.2504 - val_accuracy: 0.3806 - val_loss: 1.2639
Epoch 6/50
179/179 ━━━━━━━━━━━━━━━━━━━━ 113s 633ms/step - accuracy: 0.3786 - loss: 1.2473 - val_accuracy: 0.3829 - val_loss: 1.2460
Epoch 7/50
179/179 ━━━━━━━━━━━━━━━━━━━━ 116s 648ms/step - accuracy: 0.3869 - loss: 1.2383 - val_accuracy: 0.4462 - val_loss: 1.2447
Epoch 8/50
179/179 ━━━━━━

In [ ]:
resnet_model = create_transfer_model(ResNet50, resnet_preprocess)
resnet_model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

history_resnet = resnet_model.fit(train_generator, validation_data=test_generator, epochs=EPOCHS)
plot_history(history_resnet, 'ResNet50')
evaluate_model(resnet_model, test_generator, 'ResNet50', class_names)
show_sample_predictions(resnet_model, test_generator, class_names)

In [ ]:
inception_model = create_transfer_model(InceptionV3, inception_preprocess)
inception_model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

history_inception = inception_model.fit(train_generator, validation_data=test_generator, epochs=EPOCHS)
plot_history(history_inception, 'InceptionV3')
evaluate_model(inception_model, test_generator, 'InceptionV3', class_names)
show_sample_predictions(inception_model, test_generator, class_names)